In [1]:
import ee
import zipfile
import geopandas as gpd
import ee
import pandas as pd

ee.Initialize()
ee.Authenticate

<function ee.Authenticate(authorization_code: Optional[str] = None, quiet: Optional[bool] = None, code_verifier: Optional[str] = None, auth_mode: Optional[str] = None, scopes: Optional[Sequence[str]] = None, force: bool = False) -> Optional[bool]>

In [2]:
shapefile_path = "C:/Users/pc/My Drive/2025/Uni/TCC/Shapes/contorno_area_total/contorno_area_total.shp"

# Check if the path is a .zip file
if shapefile_path.endswith('.zip'):
    # Try to read shapefile from a zip archive
    try:
        # Check if the .zip file exists and open it
        with zipfile.ZipFile(shapefile_path, 'r') as zip_ref:
            zip_ref.printdir()  # Optional: Print contents of the zip to debug
            # Try to find the .shp file inside the zip
            shapefile_found = False
            for file in zip_ref.namelist():
                if file.endswith('.shp'):
                    shapefile_found = True
                    shapefile_within_zip = file
                    break

            if shapefile_found:
                # Read shapefile directly from the zip file
                oi = gpd.read_file(f'zip://{shapefile_path}/{shapefile_within_zip}')
                print(f"Successfully loaded shapefile from {shapefile_path}.")
            else:
                print("No .shp file found inside the zip archive.")
                #
    except Exception as e:
        print(f"Error reading shapefile from zip archive: {e}")
        #
else:
    # If not a .zip, assume it is a regular shapefile
    try:
        # Read the shapefile normally
        aoi = gpd.read_file(shapefile_path)
        print(f"Successfully loaded shapefile from {shapefile_path}.")
    except Exception as e:
        print(f"Error reading shapefile: {e}")


# After loading, check if the GeoDataFrame is not empty
if not aoi.empty:
    # If the GeoDataFrame contains multiple geometries, dissolve them into one
    if len(aoi) > 1:
        aoi = aoi.dissolve()

    # Extract the first geometry from the dissolved GeoDataFrame
    geometry = aoi.geometry.iloc[0]

    # Check if the geometry is a Polygon or MultiPolygon
    if geometry.geom_type in ['Polygon', 'MultiPolygon']:
        # Convert the geometry to GeoJSON format
        geojson = geometry.__geo_interface__

        # Remove the third dimension from the coordinates if it exists
        if geojson['type'] == 'Polygon':
            geojson['coordinates'] = [list(map(lambda coord: coord[:2], ring)) for ring in geojson['coordinates']]
        elif geojson['type'] == 'MultiPolygon':
            geojson['coordinates'] = [[list(map(lambda coord: coord[:2], ring)) for ring in polygon] for polygon in geojson['coordinates']]

        # Create an Earth Engine geometry object from the GeoJSON coordinates
        ee_geometry = ee.Geometry(geojson)

        # Convert the Earth Engine geometry to a Feature
        feature = ee.Feature(ee_geometry)

        # Create a FeatureCollection with the feature
        aoi = ee.FeatureCollection([feature])

        print("AOI defined successfully.")

        # check_next_button()
    else:
        
        print("The geometry is not a valid type (Polygon or MultiPolygon).")
else:
    print("The shapefile does not contain any geometries.")        #

Successfully loaded shapefile from C:/Users/pc/My Drive/2025/Uni/TCC/Shapes/contorno_area_total/contorno_area_total.shp.
AOI defined successfully.


In [3]:
# Define the start and end dates for filtering the image collection
inicio = '2022-01-01'
final = '2023-12-31'
nuvem ='40'

# Define the start and end dates for filtering the image collection
startDate = '2022-01-01'
endDate = '2023-12-31'

# Load the Sentinel-2 image collection and filter by date, location, and cloud coverage
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(startDate, endDate) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))

# Get the number of images in the collection
count = sentinel2.size().getInfo()
print(f"Number of images in collection: {count}")



Number of images in collection: 297


In [15]:
scl_classes_behavior = {
    0: True,  # No data
    1: True,  # Saturated/defective
    2: True,   # Dark features
    3: True,  # Cloud shadows
    4: False,   # Vegetation
    5: True,   # Bare soils
    6: False,   # Water
    7: True,   # Cloud low probability
    8: True,  # Cloud medium probability
    9: True,  # Cloud high probability
    10: True, # Thin cirrus
    11: False  # Snow or ice
}

def mask_cloud_and_shadows(image):
    scl = image.select('SCL')
    # Start with an all-inclusive mask
    mask = ee.Image.constant(1)
    # Apply exclusions using .And() method
    for class_value, include in scl_classes_behavior.items():
        if not include:
            mask = mask.And(scl.neq(class_value))
    return image.updateMask(mask)


# Apply the mask function to the collection
masked_collection = sentinel2.map(mask_cloud_and_shadows)

# Remove images with no valid pixels
def remove_empty_images(image):
    valid_pixel_count = image.mask().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=image.geometry(),
        scale=10,
        maxPixels=1e9
    ).values().get(0)
    # Check if there are valid pixels
    return ee.Image(image).set('hasValidPixels', ee.Number(valid_pixel_count).gt(0))

# Map the function to set the 'hasValidPixels' property
filtered_collection = masked_collection.map(remove_empty_images)

# Filter collection to include only images with valid pixels
filtered_collection = filtered_collection.filter(ee.Filter.eq('hasValidPixels', True))

# Get the number of images after filtering
final_count = filtered_collection.size().getInfo()
print(f"Number of images in collection after masking and filtering: {final_count}")


EEException: Collection.reduceColumns: Error in map(ID=20220119T132231_20220119T132233_T22KHV):
Image.reduceRegion: Too many pixels in the region. Found 3134571094, but maxPixels allows only 1000000000.
Ensure that you are not aggregating at a higher resolution than you intended; that is a frequent cause of this error. If not, then you may set the 'maxPixels' argument to a limit suitable for your computation; set 'bestEffort' to true to aggregate at whatever scale results in 'maxPixels' total pixels; or both.